In [23]:
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.feature import hog,local_binary_pattern
import glob
import numpy as np
from sklearn import svm
from skimage import feature
from sklearn.metrics import classification_report
from skimage.color import rgb2gray
from sklearn.preprocessing import StandardScaler

import warnings
warnings.simplefilter("ignore")

In [2]:
# Import training live and spoof data
live_train = glob.glob('data/train_live/live/*.png') #live
spoof_train = glob.glob('data/train_spoof/spoof/*.png') #spoof

# import testing live and spoof data
live_test = glob.glob('data/test_live/live/*.png') #live
spoof_test = glob.glob('data/test_spoof/spoof/*.png') #spoof

In [3]:
# Add all the images to lists
live_train_images = []
spoof_train_images = []
live_test_images = []
spoof_test_images = []

# read images from given path using sklearn imread 
# and append to the above variales
for path in live_train: live_train_images.append(imread(path))
for path in spoof_train: spoof_train_images.append(imread(path))
for path in live_test: live_test_images.append(imread(path))
for path in spoof_test: spoof_test_images.append(imread(path))

In [4]:
#Convert the above data into numpy arrays
# convert images to np arrays
live_train_images, spoof_train_images = np.asarray(live_train_images), np.asarray(spoof_train_images)
live_test_images, spoof_test_images = np.asarray(live_test_images), np.asarray(spoof_test_images)

In [18]:
def extract_features(feature_type="lbp"):
    # initialize empty lists to append features
    live_train_features, spoof_train_features = [], []
    live_test_features, spoof_test_features = [], []
    
    if(feature_type == "hog"):
    
        # calculate and store hog features of training data
        for live_img, spoof_img in zip(live_train_images, spoof_train_images):
            live_train_features.append(hog(live_img, feature_vector = True))
            spoof_train_features.append(hog(spoof_img, feature_vector = True))

        # calculate and store hog features of testing data
        for live_img, spoof_img in zip(live_test_images, spoof_test_images):
            live_test_features.append(hog(live_img, feature_vector = True))
            spoof_test_features.append(hog(spoof_img, feature_vector = True))
        
    if(feature_type == "lbp"):

        # parameters copied from scikit docs of lbp
        METHOD = 'uniform'
        R = 3
        P = 8 * R

        # calculate and store lbp features of training data
        for live_img, spoof_img in zip(live_train_images, spoof_train_images):
            live_img = rgb2gray(live_img)
            spoof_img = rgb2gray(spoof_img)
            live_train_features.append(local_binary_pattern(live_img, P, R, METHOD).flatten())
            spoof_train_features.append(local_binary_pattern(spoof_img, P, R, METHOD).flatten())

        # calculate and store lbp features of training data   
        for live_img, spoof_img in zip(live_test_images, spoof_test_images):
            # convert rgb to gray
            live_img = rgb2gray(live_img)
            # convert rgb to gray
            spoof_img = rgb2gray(spoof_img)
            live_test_features.append(local_binary_pattern(live_img, P, R, METHOD).flatten())
            spoof_test_features.append(local_binary_pattern(spoof_img, P, R, METHOD).flatten())
    
    return live_train_features, spoof_train_features, live_test_features, spoof_test_features
    

In [19]:
#Extract features using lbp first
live_train_features, spoof_train_features, live_test_features, spoof_test_features = extract_features(feature_type="lbp")

In [21]:
# define labels 1 for live and 0 for spoof
labels_train = [[1]]*len(live_train_features) + [[0]]*len(spoof_train_features)

# merge both positive and negative training data
full_train = live_train_features + spoof_train_features

# assign labels for all test data, 1 for positive, 0 for negative
labels_test = [[1]]*len(live_test_features) + [[0]]*len(spoof_test_features)

# merge both positive and negative training data
full_test = live_test_features + spoof_test_features


In [28]:
#Scale the train and test data separetely to avoid leakage
scaler = StandardScaler()
X_test = scaler.fit_transform(full_test)



In [29]:
#define the SVC classifier
clf = svm.SVC()

array([[-0.37487347, -0.50505368, -0.03851738, ...,  2.27145538,
        -0.64431606,  0.26904657],
       [ 2.12428301,  0.47563308, -0.03851738, ..., -0.2888926 ,
        -0.26474843,  0.26904657],
       [ 2.12428301, -0.50505368, -0.03851738, ..., -0.2888926 ,
        -0.13822588,  0.26904657],
       ...,
       [-0.79139955,  0.47563308, -0.03851738, ..., -0.2888926 ,
         0.11481921,  0.26904657],
       [-0.37487347,  0.47563308, -0.31864378, ..., -0.2888926 ,
        -0.64431606,  0.26904657],
       [-0.37487347,  0.47563308, -0.03851738, ..., -0.2888926 ,
        -0.51779351,  0.26904657]])

In [ ]:
# fit the model with train set
clf.fit(x_train,y_train)

In [35]:
extract_features(feat="lbp")